<a href="https://colab.research.google.com/github/jaunty0216/SomeAttempts/blob/main/%E6%96%87%E5%AD%97%E9%9B%B2%E8%A3%BD%E9%80%A0%E5%AE%B6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
!pip install jieba
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-wqy-zenhei
0 upgraded, 1 newly installed, 0 to remove and 89 not upgraded.
Need to get 7,472 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-wqy-zenhei all 0.9.45-8 [7,472 kB]
Fetched 7,472 kB in 2s (3,014 kB/s)
Selecting previously unselected package fonts-wqy-zenhei.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../fonts-wqy-zenhei_0.9.45-8_all.deb ...
Unpacking fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [48]:
from google.colab import files
uploaded = files.upload()

Saving map_comments.txt to map_comments.txt


In [80]:
import re
import jieba
from collections import Counter
import pandas as pd
from wordcloud import WordCloud
import plotly.express as px

def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))



def Paintit():
  for filename,_ in uploaded.items():
    filepath = f"/content/{filename}"
    #print(f"Filepath: {filepath}")

  OriginFile = pd.read_table(filepath,header=None,names=['contents'])
  #這邊遇到的問題：我的第一行會自動變成標頭，"header=None"解決了這問題
  #但標頭會變成'0'，"names=['contents']"解決了這問題
  Origin_df = pd.DataFrame(OriginFile)
  # print(Origin_df)

  Origin_df['Chinese_contents'] = Origin_df['contents'].apply(extract_chinese)
  Origin_df['Tokenized'] = Origin_df['Chinese_contents'].apply(lambda x: list(jieba.cut(x, HMM=True)))
  all_words = [word for tokens in Origin_df['Tokenized'] for word in tokens if len(word) > 1]

  word_counts = Counter(all_words)
  stop_words = set(['所以', '好', '因為', '大家', '的','但情', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個','節彌補'])
  filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

  word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
  word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

  # 取前 300 個最常出現的詞
  top_words = word_freq_df.head(300)

  # 使用 WQY Zenhei 字體(建立wordcloud物件)
  wordcloud = WordCloud(
      width=800,
      height=400,
      background_color='white',
      font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
  )

  # 根據詞頻生成文字雲(使用wordcloud物件執行生成作業)
  #generate_from_frequencies要接收的型態是dict
  wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

  # 將文字雲轉換為圖像(生成numpy數據)
  wordcloud_img = wordcloud.to_array()

  # 使用 Plotly 顯示文字雲
  fig = px.imshow(wordcloud_img)
  fig.update_layout(
      title="文字雲",
      xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
      yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
      coloraxis_showscale=False
  )
  fig.show()


In [81]:
Paintit()